# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_weather_df = pd.read_csv("../code/Cities_Weather.csv", encoding="utf-8")
cities_weather_df.head()

,Unnamed: 0,City,Country,Lat,Lon,Temp,Humidity,Wind Speed,Cloudiness,Date
0,0,salinopolis,BR,-0.61,-47.36,86.31,54,8.10,48,1595859255
1,1,college,US,64.86,-147.80,54.57,82,3.36,75,1595859255
2,2,poitiers,FR,46.58,0.33,85.59,45,16.11,0,1595859255
3,3,east london,ZA,-33.02,27.91,62.60,67,12.75,2,1595859255
4,4,grand-santi,GF,4.25,-54.38,87.93,55,3.40,2,1595859255


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
humidity = cities_weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_weather_df[["Lat", "Lon"]]
locations.dtypes

Lat    float64
Lon    float64
dtype: object

In [16]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = cities_weather_df.loc[(cities_weather_df["Temp"] > 70) & (cities_weather_df["Temp"] < 80) & (cities_weather_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Country,Lat,Lon,Temp,Humidity,Wind Speed,Cloudiness,Date
0,110,namibe,AO,-15.20,12.15,76.17,53,12.95,0,1595859268
1,171,mwinilunga,ZM,-11.74,24.43,79.27,20,6.58,0,1595859278
2,316,zhangye,CN,38.93,100.45,78.35,20,2.19,0,1595859300
3,424,saldanha,ZA,-33.01,17.94,73.40,33,4.70,0,1595859315
4,439,erenhot,CN,43.65,111.98,79.30,36,6.29,0,1595859316


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lon']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Country,Lat,Lon,Temp,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
0,110,namibe,AO,-15.20,12.15,76.17,53,12.95,0,1595859268,Namibe
1,171,mwinilunga,ZM,-11.74,24.43,79.27,20,6.58,0,1595859278,Mwinilunga
2,316,zhangye,CN,38.93,100.45,78.35,20,2.19,0,1595859300,Zhangye
3,424,saldanha,ZA,-33.01,17.94,73.40,33,4.70,0,1595859315,Saldanha
4,439,erenhot,CN,43.65,111.98,79.30,36,6.29,0,1595859316,Xilingol League


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))